# Advent of Code - 2024 - Day 20 - Problem 1

https://adventofcode.com/2024/day/20

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day20.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

# DATA = """###############
# #...#...#.....#
# #.#.#.#.#.###.#
# #S#...#.#.#...#
# #######.#.#.###
# #######.#.#...#
# #######.#.###.#
# ###..E#...#...#
# ###.#######.###
# #...###...#...#
# #.#####.#.###.#
# #.#...#.#.#...#
# #.#.#.#.#.#.###
# #...#...#...###
# ###############"""
# DATA = list(map(str.strip, DATA.splitlines()))

DATA

['#############################################################################################################################################',
 '#.............###.....#...#.........#...............#...#.......#...###.....#.....#...###...#...###.......###.....#.........................#',
 '#.###########.###.###.#.#.#.#######.#.#############.#.#.#.#####.#.#.###.###.#.###.#.#.###.#.#.#.###.#####.###.###.#.#######################.#',
 '#.......#...#...#...#...#.#.......#.#.......#.....#...#...#.....#.#...#...#.#...#.#.#.#...#...#...#.#.....#...#...#...#...#.............#...#',
 '#######.#.#.###.###.#####.#######.#.#######.#.###.#########.#####.###.###.#.###.#.#.#.#.#########.#.#.#####.###.#####.#.#.#.###########.#.###',
 '#...###...#...#.###.....#.........#.###...#...#...#.......#.#...#...#.#...#.....#...#.#.........#...#...###...#.....#.#.#...#...#...###.#...#',
 '#.#.#########.#.#######.###########.###.#.#####.###.#####.#.#.#.###.#.#.#############.#########.#######.#####.#####.#.#.##

## Create Map Class

In [2]:
class Map:

    def __init__(self, lines):
        self._lines = list(map(list, DATA))
        self._rows = len(self._lines)
        self._cols = len(self._lines[0])

        # Find the starting position
        #
        for row in range(self._rows):
            for col in range(self._cols):
                if self._lines[row][col] == "S":
                    self._start = (row, col)
                if self._lines[row][col] == "E":
                    self._end = (row, col)

    def can_move(self, target, cheat):
        if cheat:
            return target == "#"
        else:
            return target != "#"

    def get_next_positions(self, position, cheat):
        row, col = position
        if row > 0:
            target = self._lines[row - 1][col]
            if self.can_move(target, cheat):
                yield (row - 1, col)
        if row < self._rows - 1:
            target = self._lines[row + 1][col]
            if self.can_move(target, cheat):
                yield (row + 1, col)
        if col > 0:
            target = self._lines[row][col - 1]
            if self.can_move(target, cheat):
                yield (row, col - 1)
        if col < self._cols - 1:
            target = self._lines[row][col + 1]
            if self.can_move(target, cheat):
                yield (row, col + 1)

    def find_paths(self, path):

        last_position = path[-1]
        for next_position in self.get_next_positions(last_position, False):
            if not next_position in path:
                new_path = list(path)
                new_path.append(next_position)
                if next_position == self._end:
                    yield new_path
                else:
                    yield from self.find_paths(new_path)                

    def find_pathsx(self, path, cheat_length, path_cache):

        # print(f"find_paths - {path}")
        must_cheat = False
        can_cheat = False
        if cheat_length == len(path):
            must_cheat = True
            # print("must_cheat")
        elif cheat_length == len(path) -1:
            pass
            # can_cheat = True
            # print("can_cheat")
        cheated = len(path) > cheat_length

        last_position = path[-1]
        if cheated and last_position in path_cache:
            new_path = list(path)
            new_path.extend(path_cache[last_position])
            yield new_path
        else:
            for next_position in self.get_next_positions(last_position, must_cheat, can_cheat):
                if not next_position in path:
                    # if next_position in distances_to_exit and distances_to_exit[next_position] + len(path) > 9372:
                    #     pass
                    # else:
                    new_path = list(path)
                    new_path.append(next_position)
                    # print(next_position)
                    if next_position == self._end:
                        # print("end found")
                        yield new_path
                    else:
                        yield from self.find_paths(new_path, cheat_length, path_cache)

        # last_position = path[-1]
        # cheating = cheat_length <= len(path) <= cheat_length + 1
        # # was_cheating = cheat_length + 1 < len(path)

        # # do_pop = False
        # for next_position in self.get_next_positions(last_position, cheating):
        #     # if was_cheating:
        #     #     if next_position not in distances_to_exit: return
        #     #     if len(path) + 1 + distances_to_exit[next_position] > 10000: return

        #     if not next_position in path:
        #         new_path = list(path)
        #         new_path.append(next_position)
        #         # if do_pop: path.pop(-1)
        #         # do_pop = True
        #         # path.append(next_position)
        #         if next_position == self._end: yield new_path
        #         else: yield from self.find_paths(new_path, cheat_length, distances_to_exit)

    def print(self, path):
        m = list()
        for line in self._lines:
            m.append(list(line))

        if path != None:
            for location in path:
                row, col, is_cheat = location
                m[row][col] = "*"

        for line in m:
            print("".join(line))

## Create Path Class

In [3]:
class Path:

    def __init__(self, position):
        self._position = position

## Find Solutions

In [4]:
import sys

sys.setrecursionlimit(10000)

m = Map(DATA)
# m.print(None)
# print(m._end)

starting_path = [m._start]

# Find the path without cheating
#
for path in m.find_paths(starting_path):
    honest_path = path
    break
print(f"honest_path = {honest_path}")

honest_path_length = len(honest_path)
print(f"honest_path_length = {honest_path_length}")

lengths_to_exit = dict()
for idx in range(len(honest_path)):
    location = honest_path[idx]
    length_to_exit = honest_path_length - idx
    lengths_to_exit[location] = length_to_exit
print(f"lengths_to_exit = {lengths_to_exit}")

improvements = dict()
for idx in range(len(honest_path)):
    position = honest_path[idx]
    for position_cheat in m.get_next_positions(position, True):
        exits = set()
        for position_honest in m.get_next_positions(position_cheat, False):
            if position_honest not in exits:
                exits.add(position_honest)

                length_to_exit= lengths_to_exit[position_honest]
                cheat_length = idx + 1 + 1 + length_to_exit
                improvement = honest_path_length - cheat_length
                if improvement >= 100:
                    if improvement in improvements:
                        improvements[improvement] += 1
                    else:
                        improvements[improvement] = 1
print(f"improvements = {improvements}")

total_count = 0
for improvement in improvements:
    total_count += improvements[improvement]
print(f"total_count = {total_count}")

# print(f"honest_path = {honest_path}")

# honest_length = len(honest_path) - 1
# print(f"honest_length = {honest_length}")

# distances_to_exit = dict()
# distance_to_exit = honest_length
# for position in honest_path:
#     distances_to_exit[position] = distance_to_exit
#     distance_to_exit -= 1

# print(f"distances_to_exit = {distances_to_exit}")

# count = 0
# improvements = dict()
# for cheat_length in range(1, honest_length):  # range(1, honest_length):
#     for path in m.find_paths(starting_path, cheat_length, path_cache):
#         path_length = len(path) - 1
#         # if len(path) < honest_length - 1:
#         improvement = honest_length - path_length
#         if improvement >= 100: #== 64:
#             if improvement in improvements:
#                 improvements[improvement] += 1
#             else:
#                 improvements[improvement] = 1
#             count += 1
#             print("--------------------------------------------")
#             print(
#                 f"cheat_length = {cheat_length}, path_length={path_length}, improvement = {improvement}, count = {count}"
#             )
#             break
#             # print(path)
#             # m.print(path)

# print(count)
# print(improvements)

honest_path = [(139, 87), (139, 88), (139, 89), (139, 90), (139, 91), (139, 92), (139, 93), (138, 93), (137, 93), (137, 92), (137, 91), (137, 90), (137, 89), (136, 89), (135, 89), (135, 90), (135, 91), (134, 91), (133, 91), (133, 92), (133, 93), (132, 93), (131, 93), (131, 92), (131, 91), (130, 91), (129, 91), (129, 92), (129, 93), (128, 93), (127, 93), (127, 92), (127, 91), (127, 90), (127, 89), (127, 88), (127, 87), (126, 87), (125, 87), (124, 87), (123, 87), (123, 88), (123, 89), (122, 89), (121, 89), (120, 89), (119, 89), (118, 89), (117, 89), (116, 89), (115, 89), (114, 89), (113, 89), (112, 89), (111, 89), (111, 90), (111, 91), (110, 91), (109, 91), (108, 91), (107, 91), (106, 91), (105, 91), (104, 91), (103, 91), (102, 91), (101, 91), (100, 91), (99, 91), (99, 90), (99, 89), (99, 88), (99, 87), (98, 87), (97, 87), (97, 88), (97, 89), (96, 89), (95, 89), (95, 90), (95, 91), (96, 91), (97, 91), (97, 92), (97, 93), (98, 93), (99, 93), (100, 93), (101, 93), (102, 93), (103, 93), (10